# 감성 분석

In [47]:
import os
import re
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 1. 데이터 로드

In [48]:
## 감성 사전 ##
keywords = pd.read_excel('data/words.xlsx')

positive = list(keywords['positive'][keywords['positive'].notnull()])
negative = list(keywords['negative'][keywords['negative'].notnull()])

positive = list(set(positive))
negative = list(set(negative))

# 불용어
stopwords = ['의','가','이','은','들','는','좀','걍','과','도','를','으로','자','에','와','한','하다', '이다', '에서']

In [49]:
## 뉴스 데이터 로드 ##

# 크롤링한 뉴스 데이터의 시작, 끝 날짜
start_date = '2023.06.01'
end_date = '2023.06.15'

df = pd.read_excel('data/stock_news_result_{}.xlsx'.format(start_date+'-'+end_date), index_col = 'Unnamed: 0')

## 2. 전처리

In [50]:
## 전처리 함수 ##

regex = re.compile('[^ㄱ-ㅎ ㅏ-ㅣ 가-힣 ]')

def Pre_processing(df, stopwords):

    # 중복 행 제거
    df.drop_duplicates(['title'], inplace = True)
    df = df.reset_index(drop = True)
    
    # 정규표현식 -> 한글만 남기기
    f = lambda x: re.sub(regex, "", x)
    df['title'] = df['title'].map(f)
        
    # 토큰화
    okt = Okt()
    tokenized = []

    for i in df['title']:
        temp_x = []
        temp_x = okt.morphs(i, stem = False)
        temp_x = [word for word in temp_x if not word in stopwords] 
        tokenized.append(temp_x)
    
    return df, tokenized

In [51]:
# 전처리 수행
df_pcd, token = Pre_processing(df, stopwords)

# 토큰화된 문자열 새로운 칼럼에 넣기
df_pcd['title_tokenized'] = token

df_pcd

,stock,press,title,title_tokenized
0,삼성전자,파이낸셜뉴스,나흘간 조억 팔아치운 외국인 삼성전자는 더 담자,"[나흘, 간, 조억, 팔아치운, 외국인, 삼, 성, 전자, 더, 담자]"
1,삼성전자,한국경제,삼성전자자이대한항공하이카신한 년째 사랑받는 위,"[삼, 성, 전자, 대한항공, 하, 카신, 년, 째, 사랑받는, 위]"
2,삼성전자,전자신문언론사 선정,삼성전자 텍사스에 네트워크혁신센터 구축,"[삼성, 전자, 텍사스, 네트워크, 혁신, 센터, 구축]"
3,삼성전자,아시아경제,송봉섭 삼성전자서비스 대표 한국능률협회컨설팅 특별 공헌상 수상,"[송봉섭, 삼성, 전자, 서비스, 대표, 한국, 능률, 협회, 컨설팅, 특별, 공헌..."
4,삼성전자,대전일보,삼성전자 반도체 인재 양성 지원 확대,"[삼성, 전자, 반도체, 인재, 양성, 지원, 확대]"
...,...,...,...,...
1850,아모레퍼시픽,서울신문,서울시 아모레퍼시픽과 청년 마음건강 사업,"[서울시, 아모레퍼시픽, 청년, 마음, 건강, 사업]"
1851,아모레퍼시픽,국민일보,아모레퍼시픽재단 장원 인문학자 선발,"[아모레퍼시픽, 재단, 장원, 인문학자, 선발]"
1852,아모레퍼시픽,세계일보,제주 미래 먹거리 바이오 연구생산유통 밸류체인 구축한다,"[제주, 미래, 먹거리, 바이오, 연구, 생산, 유통, 밸류, 체인, 구축, 한다]"
1853,아모레퍼시픽,한국경제,신동원의 첫번째 깡 스낵선친 유산 잇는다,"[신동원, 첫, 번째, 깡, 스낵, 선친, 유산, 잇는다]"


## 3. 감성분석 및 스코어링(scoring)

In [52]:
# 클래스 정의
class NewsScore():
    def make(self, name):
        # 종목명
        self.name = name
        # 해당 종목의 기사
        self.news = pd.DataFrame() 
        # 스코어링 된 기사
        self.news_final = pd.DataFrame()
        # 해당 종목의 최종 스코어 
        self.score = 0 

In [53]:
# 뉴스 데이터가 있는 종목들로 최종 종목 리스트 정의
stocks_final = list(df_pcd['stock'].unique())

In [54]:
# 최종 종목별 인스턴스 생성
news_score = [NewsScore() for i in range(len(stocks_final))] 

In [55]:
for idx_s, s in enumerate(stocks_final):
    # 인스턴스에 종목명 넣기
    news_score[idx_s].make(s)
    # 해당 종목의 뉴스 df 넣기
    news_score[idx_s].news = df_pcd[df_pcd['stock'] == s] 
    news_score[idx_s].news.reset_index(drop = True, inplace = True)
    
    for t in range(len(news_score[idx_s].news)):
        # 토큰화된 제목 가져오기
        target_title = news_score[idx_s].news.loc[t, 'title_tokenized'] 
        
        for w in target_title:
            # 긍정 또는 부정 단어에 해당할 경우
            if w in positive or w in negative: 
                if w in positive:
                    # 긍정 단어일 경우 +1점
                    news_score[idx_s].score += 1 

                else:
                    # 부정 단어일 경우 -1점
                    news_score[idx_s].score -= 1 
                # 스코어링된 기사 넣기
                news_score[idx_s].news_final = pd.concat([news_score[idx_s].news_final, news_score[idx_s].news.loc[[t], :]]) 

In [56]:
# 스코어 데이터프레임 정의
df_score = pd.DataFrame({
        'stock':news_score[i].name,
        'score':news_score[i].score} for i in range(len(news_score)))

df_score.sort_values('score', ascending = False, inplace = True)
df_score.reset_index(drop = True, inplace = True)

df_score

,stock,score
0,삼성전자,39
1,카카오,18
2,SK,18
3,LG,14
4,LG에너지솔루션,10
5,삼성물산,4
6,현대차,4
7,KT&G,4
8,기업은행,4
9,KB금융,3


## 4. 결과 저장

In [57]:
# 저장할 스코어 상위 종목 개수
best_stocks_number = 5
# 저장할 스코어 하위 종목 개수
worst_stocks_number = 5 

best_stocks_list = list(df_score.loc[:best_stocks_number-1, 'stock'])
worst_stocks_list = list(df_score.loc[df_score.index[int('-'+str(worst_stocks_number))]:, 'stock'])

In [58]:
df_news_best = pd.DataFrame()
df_news_worst = pd.DataFrame()

# 스코어 상위 종목들에 해당하는 뉴스만 골라 넣기
for s in best_stocks_list:
    for i in range(len(news_score)):
        if s == news_score[i].name:
            df_news_best = pd.concat([df_news_best, news_score[i].news_final])

# 스코어 하위 종목들에 해당하는 뉴스만 골라 넣기         
for s in worst_stocks_list:
    for i in range(len(news_score)):
        if s == news_score[i].name:
            df_news_worst = pd.concat([df_news_worst, news_score[i].news_final])

In [59]:
 # 불필요한 칼럼 제거
df_news_best.drop('title_tokenized', axis = 1, inplace = True)
df_news_worst.drop('title_tokenized', axis = 1, inplace = True)

In [60]:
# 스코어 상위 종목 및 뉴스
df_news_best.to_excel('results/best_stocks_{}.xlsx'.format(start_date+'-'+end_date))

# 스코어 하위 종목 및 뉴스
df_news_worst.to_excel('results/worst_stocks_{}.xlsx'.format(start_date+'-'+end_date))

# 종목별 스코어
df_score.to_excel('results/score_{}.xlsx'.format(start_date+'-'+end_date))